In [2]:
%load_ext autoreload
%autoreload 2

from src.base_classes.omic_data_loader import OmicDataLoader
from src.data_managers.concat import CatOmicDataManager

In [25]:
mrna_loader = OmicDataLoader(
    data_dir="mds_data/splits_74/mrna",
)
mirna_loader = OmicDataLoader(
    data_dir="mds_data/splits_74/mirna_genes",
)
circrna_loader = OmicDataLoader(
    data_dir="mds_data/splits_74/circrna",
)
# pirna_loader = OmicDataLoader(
#     data_dir="mds_data/splits_74/pirna",
# )
te_loader = OmicDataLoader(
    data_dir="mds_data/splits_74/te_counts",
)

files
['train_0.csv', 'train_1.csv', 'train_2.csv', 'train_3.csv', 'train_4.csv']
['test_0.csv', 'test_1.csv', 'test_2.csv', 'test_3.csv', 'test_4.csv']
files
['train_0.csv', 'train_1.csv', 'train_2.csv', 'train_3.csv', 'train_4.csv']
['test_0.csv', 'test_1.csv', 'test_2.csv', 'test_3.csv', 'test_4.csv']
files
['train_0.csv', 'train_1.csv', 'train_2.csv', 'train_3.csv', 'train_4.csv']
['test_0.csv', 'test_1.csv', 'test_2.csv', 'test_3.csv', 'test_4.csv']
files
['train_0.csv', 'train_1.csv', 'train_2.csv', 'train_3.csv', 'train_4.csv']
['test_0.csv', 'test_1.csv', 'test_2.csv', 'test_3.csv', 'test_4.csv']


In [4]:
for fold_idx in range(5):
    train_df, test_df = mrna_loader.get_fold(fold_idx)

    print("fold: ", fold_idx)
    print(train_df["class"].value_counts(), test_df["class"].value_counts())

fold:  0
shape: (2, 2)
┌───────┬───────┐
│ class ┆ count │
│ ---   ┆ ---   │
│ i64   ┆ u32   │
╞═══════╪═══════╡
│ 0     ┆ 10    │
│ 1     ┆ 49    │
└───────┴───────┘ shape: (2, 2)
┌───────┬───────┐
│ class ┆ count │
│ ---   ┆ ---   │
│ i64   ┆ u32   │
╞═══════╪═══════╡
│ 0     ┆ 3     │
│ 1     ┆ 12    │
└───────┴───────┘
fold:  1
shape: (2, 2)
┌───────┬───────┐
│ class ┆ count │
│ ---   ┆ ---   │
│ i64   ┆ u32   │
╞═══════╪═══════╡
│ 0     ┆ 10    │
│ 1     ┆ 49    │
└───────┴───────┘ shape: (2, 2)
┌───────┬───────┐
│ class ┆ count │
│ ---   ┆ ---   │
│ i64   ┆ u32   │
╞═══════╪═══════╡
│ 0     ┆ 3     │
│ 1     ┆ 12    │
└───────┴───────┘
fold:  2
shape: (2, 2)
┌───────┬───────┐
│ class ┆ count │
│ ---   ┆ ---   │
│ i64   ┆ u32   │
╞═══════╪═══════╡
│ 0     ┆ 10    │
│ 1     ┆ 49    │
└───────┴───────┘ shape: (2, 2)
┌───────┬───────┐
│ class ┆ count │
│ ---   ┆ ---   │
│ i64   ┆ u32   │
╞═══════╪═══════╡
│ 0     ┆ 3     │
│ 1     ┆ 12    │
└───────┴───────┘
fold:  3
shape: (2, 2)
┌─

In [26]:
omic_data_loaders = {
    "mrna": mrna_loader,
    # "mirna": mirna_loader,
    # "circrna": circrna_loader,
    # "pirna": pirna_loader,
    # "te": te_loader,
}
odm = CatOmicDataManager(omic_data_loaders, n_splits=5)

In [27]:
odm.feature_dim, odm.n_classes

(200, 2)

In [ ]:
from src.evals.knn import KNNEvaluator

knn_eval = KNNEvaluator(
    data_manager=odm,
    n_trials=20,
    verbose=True,
    params={"k_lb": 1, "k_ub": 20},
)
_ = knn_eval.evaluate()
knn_eval.print_best_results()

[I 2024-11-10 19:15:05,063] A new study created in memory with name: no-name-487dc805-fc43-418d-8a2d-a156c69f7612
[I 2024-11-10 19:15:05,137] Trial 0 finished with value: 0.7155098095140549 and parameters: {'n_neighbors': 16}. Best is trial 0 with value: 0.7155098095140549.
[I 2024-11-10 19:15:05,204] Trial 1 finished with value: 0.6804361055090542 and parameters: {'n_neighbors': 12}. Best is trial 0 with value: 0.7155098095140549.
[I 2024-11-10 19:15:05,288] Trial 2 finished with value: 0.7155098095140549 and parameters: {'n_neighbors': 9}. Best is trial 0 with value: 0.7155098095140549.


New best score: 0.716
Best model performance:
Accuracy: 0.919 ± 0.050
F1 Macro: 0.852 ± 0.097
F1 Weighted: 0.913 ± 0.056


[I 2024-11-10 19:15:05,353] Trial 3 finished with value: 0.7155098095140549 and parameters: {'n_neighbors': 17}. Best is trial 0 with value: 0.7155098095140549.
[I 2024-11-10 19:15:05,421] Trial 4 finished with value: 0.7155098095140549 and parameters: {'n_neighbors': 16}. Best is trial 0 with value: 0.7155098095140549.
[I 2024-11-10 19:15:05,489] Trial 5 finished with value: 0.7155098095140549 and parameters: {'n_neighbors': 17}. Best is trial 0 with value: 0.7155098095140549.
[I 2024-11-10 19:15:05,552] Trial 6 finished with value: 0.58024233114353 and parameters: {'n_neighbors': 6}. Best is trial 0 with value: 0.7155098095140549.
[I 2024-11-10 19:15:05,615] Trial 7 finished with value: 0.7155098095140549 and parameters: {'n_neighbors': 16}. Best is trial 0 with value: 0.7155098095140549.
[I 2024-11-10 19:15:05,682] Trial 8 finished with value: 0.6804361055090542 and parameters: {'n_neighbors': 12}. Best is trial 0 with value: 0.7155098095140549.
[I 2024-11-10 19:15:05,747] Trial 9 f

Best hyperparameters:
{'n_neighbors': 16}
Best model performance:
Accuracy: 0.919 ± 0.050
F1 Macro: 0.852 ± 0.097
F1 Weighted: 0.913 ± 0.056


In [ ]:
from src.evals.svm import SVMEvaluator

svm_eval = SVMEvaluator(
    data_manager=odm,
    n_trials=30,
    verbose=True,
    params={
        "C_lb": 0.01,
        "C_ub": 10,
        "rfe_step_range": (0.05, 0.2),
        "rfe_n_features_range": (100, 200),
    },
    mode="linear",
)
_ = svm_eval.evaluate()
svm_eval.print_best_results()

[I 2024-11-10 15:04:57,374] A new study created in memory with name: no-name-17b65a7d-695e-4e4c-9283-d9a803e51944


[I 2024-11-10 15:04:57,465] Trial 0 finished with value: 0.6497992012967035 and parameters: {'C': 0.5912776084859577, 'class_weight': None, 'rfe_step': 0.18135798798684793, 'rfe_n_features': 150}. Best is trial 0 with value: 0.6497992012967035.
[I 2024-11-10 15:04:57,574] Trial 1 finished with value: 0.6497992012967035 and parameters: {'C': 1.92761937700467, 'class_weight': 'balanced', 'rfe_step': 0.12740681956658, 'rfe_n_features': 111}. Best is trial 0 with value: 0.6497992012967035.
[I 2024-11-10 15:04:57,656] Trial 2 finished with value: 0.6477728936997709 and parameters: {'C': 0.04431683340011391, 'class_weight': 'balanced', 'rfe_step': 0.0561795144594292, 'rfe_n_features': 170}. Best is trial 0 with value: 0.6497992012967035.


New best score: 0.650
Best model performance:
Accuracy: 0.892 ± 0.053
F1 Macro: 0.814 ± 0.093
F1 Weighted: 0.895 ± 0.046


[I 2024-11-10 15:04:57,742] Trial 3 finished with value: 0.5947745825537253 and parameters: {'C': 6.239037597620005, 'class_weight': None, 'rfe_step': 0.16137130043179504, 'rfe_n_features': 150}. Best is trial 0 with value: 0.6497992012967035.
[I 2024-11-10 15:04:57,804] Trial 4 finished with value: 0.6497992012967035 and parameters: {'C': 1.8024991990621209, 'class_weight': None, 'rfe_step': 0.16636180097742892, 'rfe_n_features': 200}. Best is trial 0 with value: 0.6497992012967035.
[I 2024-11-10 15:04:57,914] Trial 5 finished with value: 0.5947745825537253 and parameters: {'C': 5.247154408538435, 'class_weight': None, 'rfe_step': 0.06264027199095756, 'rfe_n_features': 156}. Best is trial 0 with value: 0.6497992012967035.
[I 2024-11-10 15:04:58,008] Trial 6 finished with value: 0.5947745825537253 and parameters: {'C': 9.321484902226024, 'class_weight': None, 'rfe_step': 0.16378265009504533, 'rfe_n_features': 154}. Best is trial 0 with value: 0.6497992012967035.
[I 2024-11-10 15:04:58,

New best score: 0.679
Best model performance:
Accuracy: 0.906 ± 0.032
F1 Macro: 0.828 ± 0.068
F1 Weighted: 0.905 ± 0.031


[I 2024-11-10 15:04:58,755] Trial 14 finished with value: 0.6787024344579722 and parameters: {'C': 0.15592096415861068, 'class_weight': 'balanced', 'rfe_step': 0.0936740719697733, 'rfe_n_features': 129}. Best is trial 12 with value: 0.6787024344579722.
[I 2024-11-10 15:04:58,870] Trial 15 finished with value: 0.7331794285989254 and parameters: {'C': 0.10243547401908666, 'class_weight': 'balanced', 'rfe_step': 0.09295207471213159, 'rfe_n_features': 131}. Best is trial 15 with value: 0.7331794285989254.
[I 2024-11-10 15:04:58,960] Trial 16 finished with value: 0.6809510924096367 and parameters: {'C': 0.050080573100068776, 'class_weight': 'balanced', 'rfe_step': 0.1057608612630386, 'rfe_n_features': 120}. Best is trial 15 with value: 0.7331794285989254.
[I 2024-11-10 15:04:59,068] Trial 17 finished with value: 0.6422617001076846 and parameters: {'C': 0.07668997260540834, 'class_weight': 'balanced', 'rfe_step': 0.0832177373698388, 'rfe_n_features': 114}. Best is trial 15 with value: 0.7331

New best score: 0.733
Best model performance:
Accuracy: 0.918 ± 0.030
F1 Macro: 0.867 ± 0.041
F1 Weighted: 0.921 ± 0.024


[I 2024-11-10 15:04:59,147] Trial 18 finished with value: 0.6477728936997709 and parameters: {'C': 0.010268450364919926, 'class_weight': 'balanced', 'rfe_step': 0.11357879132217108, 'rfe_n_features': 139}. Best is trial 15 with value: 0.7331794285989254.
[I 2024-11-10 15:04:59,238] Trial 19 finished with value: 0.6809510924096367 and parameters: {'C': 0.0644825018841236, 'class_weight': 'balanced', 'rfe_step': 0.14146501839591458, 'rfe_n_features': 116}. Best is trial 15 with value: 0.7331794285989254.
[I 2024-11-10 15:04:59,340] Trial 20 finished with value: 0.728115464597245 and parameters: {'C': 0.10675039358903013, 'class_weight': 'balanced', 'rfe_step': 0.07651257359386716, 'rfe_n_features': 139}. Best is trial 15 with value: 0.7331794285989254.
[I 2024-11-10 15:04:59,445] Trial 21 finished with value: 0.728115464597245 and parameters: {'C': 0.1155640352564498, 'class_weight': 'balanced', 'rfe_step': 0.07635708884773021, 'rfe_n_features': 138}. Best is trial 15 with value: 0.73317

Best hyperparameters:
{'C': 0.10243547401908666, 'class_weight': 'balanced', 'rfe_step': 0.09295207471213159, 'rfe_n_features': 131}
Best model performance:
Accuracy: 0.918 ± 0.030
F1 Macro: 0.867 ± 0.041
F1 Weighted: 0.921 ± 0.024


Best model performance:
Accuracy: 0.938 ± 0.058
F1 Macro: 0.684 ± 0.258
F1 Weighted: 0.924 ± 0.064

In [ ]:
from src.evals.xgboost import XGBoostEvaluator

xgb_eval = XGBoostEvaluator(
    data_manager=odm,
    n_trials=60,
    verbose=True,
)
_ = xgb_eval.evaluate()

[I 2024-11-10 19:18:41,344] A new study created in memory with name: no-name-d74618ba-7c3c-4ca8-8ef7-5eab635f3ab8
[I 2024-11-10 19:18:41,468] Trial 0 finished with value: 0.5046011800339922 and parameters: {'booster': 'gbtree', 'lambda': 1.090093549619324e-05, 'alpha': 0.0017384853973629977, 'max_depth': 6, 'eta': 0.024990822730382434, 'gamma': 0.0007448711414900602, 'grow_policy': 'lossguide'}. Best is trial 0 with value: 0.5046011800339922.
[I 2024-11-10 19:18:41,557] Trial 1 finished with value: 0.0013903478254046716 and parameters: {'booster': 'dart', 'lambda': 7.953733877278384e-07, 'alpha': 0.012311349329884858, 'max_depth': 1, 'eta': 1.3106729678135571e-08, 'gamma': 1.8912671827677967e-05, 'grow_policy': 'depthwise', 'sample_type': 'weighted', 'normalize_type': 'tree', 'rate_drop': 7.915196931299182e-06, 'skip_drop': 0.18697866330618118}. Best is trial 0 with value: 0.5046011800339922.
[I 2024-11-10 19:18:41,664] Trial 2 finished with value: 0.5947745825537253 and parameters: {'

New best score: 0.505
Best model performance:
Accuracy: 0.839 ± 0.079
F1 Macro: 0.716 ± 0.161
F1 Weighted: 0.839 ± 0.077
New best score: 0.595
Best model performance:
Accuracy: 0.879 ± 0.077
F1 Macro: 0.771 ± 0.177
F1 Weighted: 0.878 ± 0.076


[I 2024-11-10 19:18:41,755] Trial 3 finished with value: 0.4773113900356288 and parameters: {'booster': 'dart', 'lambda': 0.12408402814102056, 'alpha': 9.110488005138047e-07, 'max_depth': 2, 'eta': 0.002647083756160064, 'gamma': 0.0003482942304648263, 'grow_policy': 'depthwise', 'sample_type': 'uniform', 'normalize_type': 'forest', 'rate_drop': 0.7436722774157082, 'skip_drop': 0.43481543073977125}. Best is trial 2 with value: 0.5947745825537253.
[I 2024-11-10 19:18:41,833] Trial 4 finished with value: 0.553150014770979 and parameters: {'booster': 'gbtree', 'lambda': 0.2792310057749791, 'alpha': 0.8027307283253481, 'max_depth': 1, 'eta': 0.061263458984386496, 'gamma': 0.0005126645470834501, 'grow_policy': 'lossguide'}. Best is trial 2 with value: 0.5947745825537253.
[I 2024-11-10 19:18:41,926] Trial 5 finished with value: 0.5201471744784413 and parameters: {'booster': 'dart', 'lambda': 0.0014880441853571502, 'alpha': 3.0784878218976924e-08, 'max_depth': 2, 'eta': 0.5338883219491826, 'ga

New best score: 0.650
Best model performance:
Accuracy: 0.892 ± 0.053
F1 Macro: 0.814 ± 0.093
F1 Weighted: 0.895 ± 0.046


[I 2024-11-10 19:18:42,204] Trial 8 finished with value: 0.5844534524426352 and parameters: {'booster': 'dart', 'lambda': 0.649370961811432, 'alpha': 4.868692580741491e-07, 'max_depth': 8, 'eta': 9.401586697777986e-05, 'gamma': 1.4036135030391968e-06, 'grow_policy': 'lossguide', 'sample_type': 'weighted', 'normalize_type': 'forest', 'rate_drop': 8.381540989037945e-07, 'skip_drop': 0.04372103568102062}. Best is trial 6 with value: 0.6497992012967035.
[I 2024-11-10 19:18:42,273] Trial 9 finished with value: 0.7012351321451076 and parameters: {'booster': 'gblinear', 'lambda': 0.00014034045703319477, 'alpha': 3.3108716699093745e-05}. Best is trial 9 with value: 0.7012351321451076.
[I 2024-11-10 19:18:42,344] Trial 10 finished with value: 0.689882594756958 and parameters: {'booster': 'gblinear', 'lambda': 4.068048925310283e-08, 'alpha': 8.019133438677717e-05}. Best is trial 9 with value: 0.7012351321451076.
[I 2024-11-10 19:18:42,416] Trial 11 finished with value: 0.689882594756958 and para

New best score: 0.701
Best model performance:
Accuracy: 0.906 ± 0.053
F1 Macro: 0.851 ± 0.082
F1 Weighted: 0.910 ± 0.051


[I 2024-11-10 19:18:42,489] Trial 12 finished with value: 0.6564167377920205 and parameters: {'booster': 'gblinear', 'lambda': 1.5959550373664294e-08, 'alpha': 0.00010698868845880366}. Best is trial 9 with value: 0.7012351321451076.
[I 2024-11-10 19:18:42,562] Trial 13 finished with value: 0.689882594756958 and parameters: {'booster': 'gblinear', 'lambda': 0.00035866097780415574, 'alpha': 7.113577646025195e-05}. Best is trial 9 with value: 0.7012351321451076.
[I 2024-11-10 19:18:42,632] Trial 14 finished with value: 0.5636131189143786 and parameters: {'booster': 'gblinear', 'lambda': 5.831129710366558e-07, 'alpha': 0.001684214548016691}. Best is trial 9 with value: 0.7012351321451076.
[I 2024-11-10 19:18:42,721] Trial 15 finished with value: 0.6497992012967035 and parameters: {'booster': 'gblinear', 'lambda': 0.008447414817215762, 'alpha': 1.1290634951871592e-05}. Best is trial 9 with value: 0.7012351321451076.
[I 2024-11-10 19:18:42,790] Trial 16 finished with value: 0.565270818861396

New best score: 0.736
Best model performance:
Accuracy: 0.919 ± 0.050
F1 Macro: 0.868 ± 0.077
F1 Weighted: 0.922 ± 0.047


[I 2024-11-10 19:18:43,360] Trial 24 finished with value: 0.6712488553683833 and parameters: {'booster': 'gblinear', 'lambda': 3.3004014002630894e-05, 'alpha': 0.0006338456286067703}. Best is trial 21 with value: 0.7361252920039949.
[I 2024-11-10 19:18:43,432] Trial 25 finished with value: 0.689882594756958 and parameters: {'booster': 'gblinear', 'lambda': 0.0014121273544600171, 'alpha': 9.036017225149375e-06}. Best is trial 21 with value: 0.7361252920039949.
[I 2024-11-10 19:18:43,501] Trial 26 finished with value: 0.6787024344579722 and parameters: {'booster': 'gblinear', 'lambda': 0.032572906176680905, 'alpha': 0.0004046808096819849}. Best is trial 21 with value: 0.7361252920039949.
[I 2024-11-10 19:18:43,568] Trial 27 finished with value: 0.612600042864821 and parameters: {'booster': 'gblinear', 'lambda': 2.7072163844552095e-07, 'alpha': 0.03251672985150403}. Best is trial 21 with value: 0.7361252920039949.
[I 2024-11-10 19:18:43,690] Trial 28 finished with value: 0.558724430457101

Best hyperparameters:
{'booster': 'gblinear', 'lambda': 0.00045823825607512223, 'alpha': 1.673455869535136e-08}


In [33]:
xgb_eval.print_best_results()

Best model performance:
Accuracy: 0.919 ± 0.050
F1 Macro: 0.868 ± 0.077
F1 Weighted: 0.922 ± 0.047


In [40]:
from src.evals.mlp import MLPEvaluator

mlp_eval = MLPEvaluator(
    data_manager=odm,
    n_trials=10,
    verbose=True,
    params={
        "lr_range": [1e-4, 1e-2],
        "l2_lambda": 5e-4,
        "dropout_range": [0.1, 0.5],
        "hidden_channels": 64,
        "proj_dim": 64,
        "batch_size": 32,
        "max_epochs": 50,
    },
)
_ = mlp_eval.evaluate()

[I 2024-11-12 19:38:31,736] A new study created in memory with name: no-name-fb818b66-fdee-40b1-8f69-a558bed153e5
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/lubojjan/DiplomaGeneral/.venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
/home/lubojjan/DiplomaGeneral/.venv/lib/python3.12/site-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (2) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
`Trainer.fit` stopped: `max_epochs=50` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using:

New best score: 0.581
Best model performance:
Accuracy: 0.892 ± 0.032
F1 Macro: 0.745 ± 0.154
F1 Weighted: 0.874 ± 0.050


`Trainer.fit` stopped: `max_epochs=50` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/lubojjan/DiplomaGeneral/.venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
/home/lubojjan/DiplomaGeneral/.venv/lib/python3.12/site-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (2) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
`Trainer.fit` stopped: `max_epochs=50` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/lubojjan/DiplomaGeneral/.venv/lib/python3.12/site-pa

New best score: 0.663
Best model performance:
Accuracy: 0.906 ± 0.032
F1 Macro: 0.815 ± 0.062
F1 Weighted: 0.898 ± 0.038


`Trainer.fit` stopped: `max_epochs=50` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/lubojjan/DiplomaGeneral/.venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
/home/lubojjan/DiplomaGeneral/.venv/lib/python3.12/site-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (2) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
`Trainer.fit` stopped: `max_epochs=50` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/lubojjan/DiplomaGeneral/.venv/lib/python3.12/site-pa

New best score: 0.710
Best model performance:
Accuracy: 0.919 ± 0.026
F1 Macro: 0.846 ± 0.067
F1 Weighted: 0.913 ± 0.038


`Trainer.fit` stopped: `max_epochs=50` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/lubojjan/DiplomaGeneral/.venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
/home/lubojjan/DiplomaGeneral/.venv/lib/python3.12/site-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (2) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
`Trainer.fit` stopped: `max_epochs=50` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/lubojjan/DiplomaGeneral/.venv/lib/python3.12/site-pa

In [41]:
mlp_eval.print_best_results()
mlp_eval.print_best_parameters()

Best model performance:
Accuracy: 0.919 ± 0.026
F1 Macro: 0.846 ± 0.067
F1 Weighted: 0.913 ± 0.038
Best hyperparameters:
{'lr': 0.007039917108469391, 'dropout': 0.44376947156025015}


In [92]:
knn_eval.save_results(results_file="logs/mds_disese_eval.csv", row_name="knn")
svm_eval.save_results(results_file="logs/mds_disese_eval.csv", row_name="svm")
xgb_eval.save_results(results_file="logs/mds_disese_eval.csv", row_name="xgb")

In [127]:
knn_eval.print_best_results()
svm_eval.print_best_results()
xgb_eval.print_best_results()

Best model performance:
Accuracy: 0.923 ± 0.084
F1 Macro: 0.825 ± 0.208
F1 Weighted: 0.933 ± 0.072
Best model performance:
Accuracy: 0.954 ± 0.038
F1 Macro: 0.688 ± 0.255
F1 Weighted: 0.932 ± 0.056
Best model performance:
Accuracy: 0.954 ± 0.038
F1 Macro: 0.688 ± 0.255
F1 Weighted: 0.932 ± 0.056


In [82]:
mrna_loader.get_fold(0)

(shape: (59, 202)
 ┌────────────┬────────────┬────────────┬───────────┬───┬───────────┬───────────┬───────────┬───────┐
 │ ENSG000001 ┆ ENSG000002 ┆ ENSG000001 ┆ ENSG00000 ┆ … ┆ ENSG00000 ┆ ENSG00000 ┆ sample_id ┆ class │
 │ 81826      ┆ 78588      ┆ 20594      ┆ 121797    ┆   ┆ 130821    ┆ 172159    ┆ s         ┆ ---   │
 │ ---        ┆ ---        ┆ ---        ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ i64   │
 │ f64        ┆ f64        ┆ f64        ┆ f64       ┆   ┆ f64       ┆ f64       ┆ str       ┆       │
 ╞════════════╪════════════╪════════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════╡
 │ 0.190054   ┆ 0.8589     ┆ 0.724728   ┆ 0.796341  ┆ … ┆ 0.235535  ┆ 0.200897  ┆ N54       ┆ 0     │
 │ 0.131677   ┆ 0.788182   ┆ 0.73847    ┆ 0.566229  ┆ … ┆ 0.461213  ┆ 0.061081  ┆ N58       ┆ 0     │
 │ 0.94058    ┆ 0.623225   ┆ 0.735662   ┆ 0.41357   ┆ … ┆ 0.114601  ┆ 0.226098  ┆ N82       ┆ 0     │
 │ 0.805491   ┆ 0.642498   ┆ 0.62087    ┆ 0.43411   ┆ … ┆ 0.0   

In [87]:
odm.get_split(0)

(shape: (59, 800)
 ┌────────────┬────────────┬────────────┬───────────┬───┬──────────┬──────────┬──────────┬──────────┐
 │ ENSG000001 ┆ ENSG000002 ┆ ENSG000001 ┆ ENSG00000 ┆ … ┆ L1M3DE_5 ┆ HERV19I  ┆ LTR40C   ┆ L1MA8    │
 │ 81826      ┆ 78588      ┆ 20594      ┆ 121797    ┆   ┆ ---      ┆ ---      ┆ ---      ┆ ---      │
 │ ---        ┆ ---        ┆ ---        ┆ ---       ┆   ┆ f64      ┆ f64      ┆ f64      ┆ f64      │
 │ f64        ┆ f64        ┆ f64        ┆ f64       ┆   ┆          ┆          ┆          ┆          │
 ╞════════════╪════════════╪════════════╪═══════════╪═══╪══════════╪══════════╪══════════╪══════════╡
 │ 0.190054   ┆ 0.8589     ┆ 0.724728   ┆ 0.796341  ┆ … ┆ 0.770044 ┆ 0.741842 ┆ 0.767158 ┆ 0.87709  │
 │ 0.131677   ┆ 0.788182   ┆ 0.73847    ┆ 0.566229  ┆ … ┆ 0.628628 ┆ 0.630266 ┆ 0.27247  ┆ 0.548661 │
 │ 0.94058    ┆ 0.623225   ┆ 0.735662   ┆ 0.41357   ┆ … ┆ 0.417187 ┆ 0.635949 ┆ 0.342004 ┆ 0.649414 │
 │ 0.805491   ┆ 0.642498   ┆ 0.62087    ┆ 0.43411   ┆ … ┆ 0.7912

In [ ]:
catodm = CatOmicDataManager(
    omic_data_loaders={
        "mrna": mrna_loader,
        # "meth": meth_loader,
        "mirna": mirna_loader,
    },
    n_splits=5,
)

catodm.get_split(0)

(shape: (59, 400)
 ┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
 │ ENSG00000 ┆ ENSG00000 ┆ ENSG00000 ┆ ENSG00000 ┆ … ┆ ENSG00000 ┆ ENSG00000 ┆ ENSG00000 ┆ ENSG0000 │
 │ 181826    ┆ 278588    ┆ 120594    ┆ 121797    ┆   ┆ 276404    ┆ 207820    ┆ 252695    ┆ 0263831  │
 │ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
 │ f64       ┆ f64       ┆ f64       ┆ f64       ┆   ┆ f64       ┆ f64       ┆ f64       ┆ f64      │
 ╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
 │ 0.190054  ┆ 0.8589    ┆ 0.724728  ┆ 0.796341  ┆ … ┆ 0.792052  ┆ 0.724515  ┆ 0.670447  ┆ 0.877857 │
 │ 0.131677  ┆ 0.788182  ┆ 0.73847   ┆ 0.566229  ┆ … ┆ 0.570109  ┆ 0.788338  ┆ 0.25199   ┆ 0.445283 │
 │ 0.94058   ┆ 0.623225  ┆ 0.735662  ┆ 0.41357   ┆ … ┆ 0.39374   ┆ 0.692776  ┆ 0.478255  ┆ 0.630045 │
 │ 0.805491  ┆ 0.642498  ┆ 0.62087   ┆ 0.43411   ┆ … ┆ 0.656948 

In [167]:
from src.evals.mogonet import MOGONETEvaluator
from src.data_managers.sample_graph import SampleGraphDataManager

mogonet_eval = MOGONETEvaluator(
    data_manager=SampleGraphDataManager(
        omic_data_loaders={
            "mrna": mrna_loader,
            # "mirna": mirna_loader,
        },
        n_splits=5,
        params={
            "graph_style": "threshold",
            "self_connections": True,
            "avg_degree": 5,
        },
    ),
    n_trials=1,
    params={
        "encoder_hidden_channels": {
            "mrna": 64,
            # "mirna": 64,
        },
        "encoder_type": "gat",
        "dropout": 0.2,
        "integrator_type": "linear",
        "integration_in_dim": 16,
        "vcdn_hidden_channels": 16,
        "epochs": 100,
        "log_interval": 25,
    }
)
mogonet_eval.evaluate()

[I 2024-11-12 23:34:09,314] A new study created in memory with name: no-name-f369a223-3c9c-4fe4-9dfd-0047230ac178



Epoch: 025:
Train Loss: 0.5289, Train Acc: 0.6780, Train F1 Macro: 0.4040, Train F1 Weighted: 0.6711
Val Acc: 0.8000, Val F1 Macro: 0.4444, Val F1 Weighted: 0.7111
Test Acc: 0.8000, Test F1 Macro: 0.4444, Test F1 Weighted: 0.7111
##################################################

Epoch: 050:
Train Loss: 0.3526, Train Acc: 0.7458, Train F1 Macro: 0.5295, Train F1 Weighted: 0.7404
Val Acc: 0.8000, Val F1 Macro: 0.4444, Val F1 Weighted: 0.7111
Test Acc: 0.8000, Test F1 Macro: 0.4444, Test F1 Weighted: 0.7111
##################################################

Epoch: 075:
Train Loss: 0.2607, Train Acc: 0.9153, Train F1 Macro: 0.8552, Train F1 Weighted: 0.9168
Val Acc: 0.8667, Val F1 Macro: 0.7115, Val F1 Weighted: 0.8385
Test Acc: 0.8667, Test F1 Macro: 0.7115, Test F1 Weighted: 0.8385
##################################################

Epoch: 100:
Train Loss: 0.1916, Train Acc: 0.9322, Train F1 Macro: 0.8796, Train F1 Weighted: 0.9322
Val Acc: 0.8667, Val F1 Macro: 0.7115, Val F1 Weight

[I 2024-11-12 23:34:11,036] Trial 0 finished with value: 0.6882308988782632 and parameters: {}. Best is trial 0 with value: 0.6882308988782632.



Epoch: 075:
Train Loss: 0.3202, Train Acc: 0.8500, Train F1 Macro: 0.7726, Train F1 Weighted: 0.8566
Val Acc: 0.9286, Val F1 Macro: 0.8783, Val F1 Weighted: 0.9342
Test Acc: 0.9286, Test F1 Macro: 0.8783, Test F1 Weighted: 0.9342
##################################################

Epoch: 100:
Train Loss: 0.2251, Train Acc: 0.8667, Train F1 Macro: 0.8137, Train F1 Weighted: 0.8766
Val Acc: 0.8571, Val F1 Macro: 0.7879, Val F1 Weighted: 0.8745
Test Acc: 0.8571, Test F1 Macro: 0.7879, Test F1 Weighted: 0.8745
##################################################
New best score: 0.688
Best model performance:
Accuracy: 0.905 ± 0.055
F1 Macro: 0.842 ± 0.096
F1 Weighted: 0.904 ± 0.056


{'acc': np.float64(0.9047619047619048),
 'f1_macro': np.float64(0.8417925407925407),
 'f1_weighted': np.float64(0.9036386280386282),
 'acc_std': np.float64(0.05487565825100165),
 'f1_macro_std': np.float64(0.09644917742011722),
 'f1_weighted_std': np.float64(0.055988681352628106)}

In [98]:
trainer.test(data, data.test_mask)

(0.32602688670158386, tensor([1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]))

In [65]:
data.train_mask, data.test_mask

(tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0]),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1]))

In [67]:
data.y

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1])

In [96]:
a = torch.tensor([1, 2, 3, 4, 5])
b = torch.tensor([0, 0, 1, 1, 1])
a[b], a[b.bool()]

(tensor([1, 1, 2, 2, 2]), tensor([3, 4, 5]))

In [70]:
data.y[data.test_mask]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0])